# PHASE 2: Data Ingestion & Understanding

## Overview

This notebook covers:
1. **Time-Series Data**: NASA C-MAPSS turbofan engine degradation dataset
2. **Text Data**: LogHub system logs (HDFS, BGL)

We will:
- Download and parse the C-MAPSS dataset
- Visualize sensor degradation patterns
- Create RUL (Remaining Useful Life) labels
- Split data by engine (train/val/test)
- Download and normalize LogHub datasets
- Create incident narratives from logs
- Generate synthetic maintenance reports

**Timeline**: Days 4–7 of project

# Section 1: Setup & Imports

In [9]:
import warnings
warnings.filterwarnings('ignore')

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure Visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ==================== 1. AUTOMATIC DATASET DETECTION ====================
print("🔍 Scanning for datasets...")

# Define potential search paths (Project structure + User's Local Path)
SEARCH_PATHS = [
    Path("data/raw/CMAPSS"),                            # Standard project path
    Path("/Users/xe/Desktop/CAPSTONE KRISHNA SIR /CMaps/"), # User's local path
    Path("data/raw"),                                   # Fallback
    Path(".")                                           # Current dir
]

DATA_DIR = None
for p in SEARCH_PATHS:
    if p.exists() and list(p.glob(f"train_FD001.txt")):
        DATA_DIR = p
        print(f"✅ Found C-MAPSS data in: {DATA_DIR}")
        break

if DATA_DIR is None:
    raise FileNotFoundError("❌ Could not locate C-MAPSS files (train_FD001.txt, etc). Please check uploads.")

# Derive absolute project root from DATA_DIR: .../data/raw/CMAPSS → project root
PROJECT_ROOT     = DATA_DIR.resolve().parent.parent.parent
INTERIM_DATA_DIR = PROJECT_ROOT / 'data' / 'interim'
ED_REPORT_DIR    = PROJECT_ROOT / 'reports' / 'figures' / 'eda'
INTERIM_DATA_DIR.mkdir(parents=True, exist_ok=True)
ED_REPORT_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Project Root resolved to: {PROJECT_ROOT}")

# Scan for Text/Log Data
TEXT_DATA_FILES = []
for p in SEARCH_PATHS:
    if p.exists():
        logs = list(p.glob("*log*.txt")) + list(p.glob("*maintenance*.txt")) + list(p.glob("*doc*.txt"))
        if logs:
            TEXT_DATA_FILES.extend(logs)

if TEXT_DATA_FILES:
    print(f"✅ Found extra text datasets: {[f.name for f in TEXT_DATA_FILES]}")
else:
    print("⚠️ No specific 'maintenance/log' text files found. (Will check data/raw/sample_logs.txt)")

# ==================== 2. CONFIGURATION ====================
# Columns based on C-MAPSS documentation (26 columns)
# 1-2: Metadata (Engine, Cycle)
# 3-5: Operational Settings
# 6-26: Sensor Readings (s1 - s21)
COL_NAMES = ['engine_id', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + \
            [f'sensor_{i}' for i in range(1, 22)]

print(f"\n📋 Schema Defined: {len(COL_NAMES)} columns")
print(f"   Metadata: {COL_NAMES[:2]}")
print(f"   Sensors:  {COL_NAMES[5:]} (Count: {len(COL_NAMES[5:])})")
print(f"📂 EDA reports  → {ED_REPORT_DIR}")
print(f"📂 Interim data → {INTERIM_DATA_DIR}")


🔍 Scanning for datasets...
✅ Found C-MAPSS data in: /Users/xe/Desktop/CAPSTONE KRISHNA SIR /CMaps
✅ Project Root resolved to: /Users/xe
⚠️ No specific 'maintenance/log' text files found. (Will check data/raw/sample_logs.txt)

📋 Schema Defined: 26 columns
   Metadata: ['engine_id', 'cycle']
   Sensors:  ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21'] (Count: 21)
📂 EDA reports  → /Users/xe/reports/figures/eda
📂 Interim data → /Users/xe/data/interim


In [10]:
# ==================== 3. DATASET LOADING & BASIC INFO ====================
print("\nLoading datasets...")

# Helper to load space-separated file (C-MAPSS format)
def load_cmapss_file(fpath):
    print(f"   Reading {fpath.name}...")
    try:
        # Load, use header=None, assign columns
        df = pd.read_csv(fpath, sep=r"\s+", header=None)
        
        # Infer headers if columns doesn't match
        if df.shape[1] == 26:
            df.columns = COL_NAMES
        elif df.shape[1] == 28: # Some versions have trailing empty cols
             df = df.iloc[:, :26]
             df.columns = COL_NAMES
        # Handle RUL files (Single column)
        elif df.shape[1] == 1:
            df.columns = ['RUL_end']
        
        return df
    except Exception as e:
        print(f"❌ Error loading {fpath.name}: {e}")
        return pd.DataFrame()

# Dictionary to hold all DataFrames { 'train_FD001': df, 'test_FD001': df, ... }
datasets = {}

for subset in ['FD001', 'FD002', 'FD003', 'FD004']:
    train_f = DATA_DIR / f'train_{subset}.txt'
    test_f = DATA_DIR / f'test_{subset}.txt'
    rul_f = DATA_DIR / f'RUL_{subset}.txt'
    
    if train_f.exists():
        d_train = load_cmapss_file(train_f)
        d_train['subset'] = subset # Track source
        datasets[f'train_{subset}'] = d_train
        
    if test_f.exists():
        d_test = load_cmapss_file(test_f)
        d_test['subset'] = subset
        datasets[f'test_{subset}'] = d_test
        
    if rul_f.exists():
        d_rul = load_cmapss_file(rul_f)
        d_rul['subset'] = subset
        # Add engine_id (1-indexed based on row number)
        d_rul['engine_id'] = d_rul.index + 1
        datasets[f'rul_{subset}'] = d_rul

print(f"\n✅ Total Datasets Loaded: {len(datasets)}")

# ==================== 4. BASIC STATISTICS ====================
summary_rows = []

print("\n--- DATASET OVERVIEW ---")
for ds_name, df in datasets.items():
    if 'rul' in ds_name: continue # Skip RUL files for generic stats
    
    # 1. Unique Engines
    n_engines = df['engine_id'].nunique()
    
    # 2. Cycle Stats
    cycle_stats = df.groupby('engine_id')['cycle'].agg(['min', 'max', 'count'])
    avg_cycles = cycle_stats['max'].mean()
    min_cycles = cycle_stats['max'].min()
    max_cycles = cycle_stats['max'].max()
    
    # 3. Missing/Dupe
    missing = df.isnull().sum().sum()
    dupes = df.duplicated().sum()
    
    print(f"Dataset: {ds_name}")
    print(f"   Shape: {df.shape}")
    print(f"   Engines: {n_engines}")
    print(f"   Cycle Length (Avg/Min/Max): {avg_cycles:.1f} / {min_cycles} / {max_cycles}")
    
    if missing > 0: print(f"   ⚠️ WARNING: {missing} missing values found!")
    if dupes > 0:   print(f"   ⚠️ WARNING: {dupes} duplicate rows found!")
    
    summary_rows.append({
        'Dataset': ds_name,
        'Rows': len(df),
        'Cols': df.shape[1],
        'Engines': n_engines,
        'Avg_Cycle': avg_cycles,
        'Min_Cycle': min_cycles,
        'Max_Cycle': max_cycles
    })

# Save Summary
pd.DataFrame(summary_rows).to_csv(ED_REPORT_DIR / "dataset_summary.csv", index=False)
print(f"\n✅ Summary saved to: {ED_REPORT_DIR}/dataset_summary.csv")



Loading datasets...
   Reading train_FD001.txt...
   Reading test_FD001.txt...
   Reading RUL_FD001.txt...
   Reading train_FD002.txt...
   Reading test_FD002.txt...
   Reading RUL_FD002.txt...
   Reading train_FD003.txt...
   Reading test_FD003.txt...
   Reading RUL_FD003.txt...
   Reading train_FD004.txt...
   Reading test_FD004.txt...
   Reading RUL_FD004.txt...

✅ Total Datasets Loaded: 12

--- DATASET OVERVIEW ---
Dataset: train_FD001
   Shape: (20631, 27)
   Engines: 100
   Cycle Length (Avg/Min/Max): 206.3 / 128 / 362
Dataset: test_FD001
   Shape: (13096, 27)
   Engines: 100
   Cycle Length (Avg/Min/Max): 131.0 / 31 / 303
Dataset: train_FD002
   Shape: (53759, 27)
   Engines: 260
   Cycle Length (Avg/Min/Max): 206.8 / 128 / 378
Dataset: test_FD002
   Shape: (33991, 27)
   Engines: 259
   Cycle Length (Avg/Min/Max): 131.2 / 21 / 367
Dataset: train_FD003
   Shape: (24720, 27)
   Engines: 100
   Cycle Length (Avg/Min/Max): 247.2 / 145 / 525
Dataset: test_FD003
   Shape: (16596, 27

# Section 2: Download and Parse C-MAPSS Time-Series Dataset

## Step 1: Check Dataset Files

First, let's check if the C-MAPSS dataset is already downloaded. If not, we can use the download script.

**To download the dataset:**
1. Set up Kaggle credentials: `~/.kaggle/kaggle.json`
2. Run: `python scripts/download_cmapss.py`

For now, we'll assume the files are available at `data/raw/CMAPSS/`

## Step 2: Load and Parse C-MAPSS Data

We'll load all four C-MAPSS subsets (FD001–FD004).

Dataset structure:
- **FD001**: 100 engines, 1 operating condition, 1 fault mode
- **FD002**: 260 engines, 6 operating conditions, 1 fault mode
- **FD003**: 100 engines, 1 operating condition, 2 fault modes
- **FD004**: 248 engines, 6 operating conditions, 2 fault modes

## Step 3: Explore Dataset Statistics

In [11]:
# ==================== 5. SENSOR ANALYSIS & PLOTS ====================
print("\n--- SENSOR ANALYSIS ---")

# Combine all Train Data for global analysis (with subset ID)
train_dfs = [datasets[k] for k in datasets if 'train' in k]
full_train_df = pd.concat(train_dfs, ignore_index=True)

test_dfs = [datasets[k] for k in datasets if 'test' in k]
full_test_df = pd.concat(test_dfs, ignore_index=True)

# 1. Identify Constant Sensors
# Some sensors in C-MAPSS are constant (variance = 0), especially in FD001/003 (single condition)
sensor_cols = [c for c in full_train_df.columns if 'sensor' in c]
variances = full_train_df[sensor_cols].var()
constant_sensors = variances[variances < 1e-5].index.tolist()

print(f"📉 Analyzing Sensor Variance (Global Train Set)...")
if constant_sensors:
    print(f"⚠️ Found {len(constant_sensors)} Constant/Near-Constant Sensors (Global): {constant_sensors}")
else:
    print("✅ No globally constant sensors found.")

# Check Per-Subset Constancy (Important because FD001 is simpler than FD002)
print("\n--- Per-Subset Constant Sensors ---")
for subset in ['FD001', 'FD002', 'FD003', 'FD004']:
    sub_df = full_train_df[full_train_df['subset'] == subset]
    sub_var = sub_df[sensor_cols].var()
    const_sub = sub_var[sub_var < 1e-5].index.tolist()
    if const_sub:
        print(f"   {subset}: {const_sub}")
    else:
        print(f"   {subset}: None")

# 2. Cycle Length Distribution Plot
plt.figure(figsize=(15, 5))
for subset in ['FD001', 'FD002', 'FD003', 'FD004']:
    # Get max cycle per engine for this subset
    max_cycles = datasets[f'train_{subset}'].groupby('engine_id')['cycle'].max()
    sns.kdeplot(max_cycles, label=subset, clip=(0, None))
    
plt.title("Engine Life Duration (Max Cycles) Distribution by Dataset")
plt.xlabel("Max Cycles")
plt.legend()
plt.savefig(ED_REPORT_DIR / "cycle_distribution.png")
plt.close()
print(f"✅ Saved plot: {ED_REPORT_DIR}/cycle_distribution.png")

# 3. Sensor Correlation Heatmap (FD001 as baseline)
# We use FD001 because generally it has the clearest degradation signal without multiple Op Conditions noise
plt.figure(figsize=(12, 10))
corr = datasets['train_FD001'][sensor_cols].corr()
sns.heatmap(corr, cmap='coolwarm', vmin=-1, vmax=1, annot=False)
plt.title("Sensor Correlation Matrix (FD001)")
plt.tight_layout()
plt.savefig(ED_REPORT_DIR / "correlation_heatmap_FD001.png")
plt.close()
print(f"✅ Saved plot: {ED_REPORT_DIR}/correlation_heatmap_FD001.png")

# 4. Train vs Test Distribution (Drift Check)
# Pick a highly variable sensor (e.g., Sensor 11 or 12)
sensor_metric = 'sensor_11'
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.kdeplot(full_train_df[sensor_metric], label='Train (All)', shade=True, color='blue')
sns.kdeplot(full_test_df[sensor_metric], label='Test (All)', shade=True, color='orange')
plt.title(f"{sensor_metric} Distribution: Train vs Test")
plt.legend()

plt.subplot(1, 2, 2)
# Check drift per subset
for subset in ['FD001', 'FD002', 'FD003', 'FD004']:
    sns.kdeplot(datasets[f'train_{subset}'][sensor_metric], label=f'{subset}', alpha=0.5)
plt.title(f"{sensor_metric} Distribution by Subset")
plt.legend()

plt.tight_layout()
plt.savefig(ED_REPORT_DIR / "sensor_drift_check.png")
plt.close()
print(f"✅ Saved plot: {ED_REPORT_DIR}/sensor_drift_check.png")



--- SENSOR ANALYSIS ---
📉 Analyzing Sensor Variance (Global Train Set)...
✅ No globally constant sensors found.

--- Per-Subset Constant Sensors ---
   FD001: ['sensor_1', 'sensor_5', 'sensor_6', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']
   FD002: None
   FD003: ['sensor_1', 'sensor_5', 'sensor_16', 'sensor_18', 'sensor_19']
   FD004: None
✅ Saved plot: /Users/xe/reports/figures/eda/cycle_distribution.png
✅ Saved plot: /Users/xe/reports/figures/eda/correlation_heatmap_FD001.png
✅ Saved plot: /Users/xe/reports/figures/eda/sensor_drift_check.png


# Section 3: Merge, Save & Text Data

In [12]:
# ==================== 6. TEXT DATA EDA (LogHub / Maintenance Logs) ====================
print("\n--- TEXT DATA EDA ---")

combined_text_df = pd.DataFrame(columns=['source_file', 'line_content'])

if TEXT_DATA_FILES:
    for f in TEXT_DATA_FILES:
        try:
            # Read first few lines
            with open(f, 'r', encoding='utf-8', errors='ignore') as log_file:
                lines = log_file.readlines()
                
            print(f"File: {f.name}")
            print(f"   Shape: {len(lines)} lines")
            print(f"   Preview (First 3 lines):")
            for i, line in enumerate(lines[:3]):
                print(f"     {i+1}: {line.strip()}")
            
            # Simple DataFrame construction
            temp_df = pd.DataFrame({'line_content': lines})
            temp_df['source_file'] = f.name
            combined_text_df = pd.concat([combined_text_df, temp_df], ignore_index=True)
            
        except Exception as e:
            print(f"❌ Error reading text file {f.name}: {e}")
else:
    print("⚠️ No text log files processed.")

# Save Combined Text
if not combined_text_df.empty:
    combined_text_df.to_csv(INTERIM_DATA_DIR / "combined_text_corpus.csv", index=False)
    print(f"✅ Combined text corpus saved: {INTERIM_DATA_DIR}/combined_text_corpus.csv")

# ==================== 7. MERGE & SAVE C-MAPSS DATASETS ====================
print("\n--- MERGING & SAVING ---")

# Ensure 'subset' col exists (added during loading)
# Also add 'dataset_id' as requested (integer or string)
# Let's use string 'FD001' etc as subset, and map to int dataset_id if needed.
# User asked for 'dataset_id'. I'll map FD001->1, FD002->2 etc.

dataset_map = {'FD001': 1, 'FD002': 2, 'FD003': 3, 'FD004': 4}

# Process TRAIN
full_train_df = pd.concat([datasets[k] for k in datasets if 'train' in k], ignore_index=True)
full_train_df['dataset_id'] = full_train_df['subset'].map(dataset_map)
full_train_df.to_csv(INTERIM_DATA_DIR / "combined_train.csv", index=False)
print(f"✅ Saved Combined Train: {INTERIM_DATA_DIR}/combined_train.csv (Shape: {full_train_df.shape})")

# Process TEST
full_test_df = pd.concat([datasets[k] for k in datasets if 'test' in k], ignore_index=True)
full_test_df['dataset_id'] = full_test_df['subset'].map(dataset_map)
full_test_df.to_csv(INTERIM_DATA_DIR / "combined_test.csv", index=False)
print(f"✅ Saved Combined Test:  {INTERIM_DATA_DIR}/combined_test.csv  (Shape: {full_test_df.shape})")

# Process RUL (For Evaluation Later)
full_rul_df = pd.concat([datasets[k] for k in datasets if 'rul' in k], ignore_index=True)
full_rul_df['dataset_id'] = full_rul_df['subset'].map(dataset_map)
full_rul_df.to_csv(INTERIM_DATA_DIR / "combined_test_rul.csv", index=False)
print(f"✅ Saved Combined RUL:   {INTERIM_DATA_DIR}/combined_test_rul.csv   (Shape: {full_rul_df.shape})")

print("\n🎉 EDA PHASE COMPLETE. Proceed to Feature Engineering.")



--- TEXT DATA EDA ---
⚠️ No text log files processed.

--- MERGING & SAVING ---
✅ Saved Combined Train: /Users/xe/data/interim/combined_train.csv (Shape: (160359, 28))
✅ Saved Combined Test:  /Users/xe/data/interim/combined_test.csv  (Shape: (104897, 28))
✅ Saved Combined RUL:   /Users/xe/data/interim/combined_test_rul.csv   (Shape: (707, 4))

🎉 EDA PHASE COMPLETE. Proceed to Feature Engineering.


In [13]:
# ==================== ASSEMBLE full_rul_df FROM ALL FOUR RUL FILES ====================
# Combines rul_FD001 … rul_FD004 into a single DataFrame used by the RUL-label cell below.
# dataset_map is defined in Cell 8: {'FD001': 1, 'FD002': 2, 'FD003': 3, 'FD004': 4}

rul_pieces = []
for subset in ['FD001', 'FD002', 'FD003', 'FD004']:
    key = f'rul_{subset}'
    if key in datasets:
        df = datasets[key][['engine_id', 'RUL_end', 'subset']].copy()
        df['dataset_id'] = dataset_map[subset]   # FD001→1, FD002→2, etc.
        rul_pieces.append(df)
    else:
        print(f"⚠️ Key '{key}' not found in datasets — check that RUL_{subset}.txt loaded correctly.")

if not rul_pieces:
    raise ValueError("❌ No RUL data assembled. Ensure all four RUL_FDxxx.txt files exist in DATA_DIR.")

full_rul_df = pd.concat(rul_pieces, ignore_index=True)
print(f"✅ full_rul_df: {full_rul_df.shape}  engines per subset: "
      f"{full_rul_df.groupby('dataset_id').size().to_dict()}")
print(full_rul_df.head())


✅ full_rul_df: (707, 4)  engines per subset: {1: 100, 2: 259, 3: 100, 4: 248}
   engine_id  RUL_end subset  dataset_id
0          1      112  FD001           1
1          2       98  FD001           1
2          3       69  FD001           1
3          4       82  FD001           1
4          5       91  FD001           1


# Section 4: Create Remaining Useful Life (RUL) Labels

In [ ]:

import pandas as pd
import numpy as np

# =============================================================================
# RUL LABELS FOR ALL 4 C-MAPSS SUBSETS (FD001–FD004)
# =============================================================================
# Composite key [dataset_id, engine_id] ensures FD001 engine 1 ≠ FD002 engine 1.
# Idempotent: safe to re-run without creating duplicate columns.

# ---------- 1. TRAIN RUL (run-to-failure) ----------
# Drop any leftover columns from a previous run
for col in ['max_cycle', 'RUL', 'RUL_clipped']:
    if col in full_train_df.columns:
        full_train_df.drop(columns=[col], inplace=True)

max_cycle = (full_train_df
             .groupby(['dataset_id', 'engine_id'])['cycle']
             .transform('max'))
full_train_df['max_cycle']   = max_cycle
full_train_df['RUL']         = full_train_df['max_cycle'] - full_train_df['cycle']
full_train_df['RUL_clipped'] = full_train_df['RUL'].clip(upper=125)

# ---------- 2. TEST RUL (truncated — uses official RUL_FDxxx.txt) ----------
for col in ['max_cycle', 'RUL', 'RUL_clipped', 'RUL_end']:
    if col in full_test_df.columns:
        full_test_df.drop(columns=[col], inplace=True)

max_cycle_test = (full_test_df
                  .groupby(['dataset_id', 'engine_id'])['cycle']
                  .transform('max'))
full_test_df['max_cycle'] = max_cycle_test

# Merge with official RUL — use dataset_id + engine_id, drop 'subset' from rul to
# avoid duplicate subset_x / subset_y columns
rul_merge = full_rul_df[['dataset_id', 'engine_id', 'RUL_end']].copy()
full_test_df = full_test_df.merge(rul_merge, on=['dataset_id', 'engine_id'], how='left')

full_test_df['RUL']         = full_test_df['RUL_end'] + (full_test_df['max_cycle'] - full_test_df['cycle'])
full_test_df['RUL_clipped'] = full_test_df['RUL'].clip(upper=125)

# ---------- 3. Summary ----------
print("📊 RUL Labels — ALL 4 C-MAPSS Subsets Combined")
print(f"   Train: {full_train_df.shape}  subsets={sorted(full_train_df['subset'].unique())}")
print(f"   Test:  {full_test_df.shape}   subsets={sorted(full_test_df['subset'].unique())}")
print(f"\nTrain RUL stats:")
print(full_train_df[['RUL', 'RUL_clipped']].describe().round(1))
print(f"\nTest RUL stats:")
print(full_test_df[['RUL', 'RUL_clipped']].describe().round(1))

# Per-subset engine counts
print("\n📊 Engines per subset:")
for split_name, df_ in [('Train', full_train_df), ('Test', full_test_df)]:
    counts = df_.groupby('subset')['engine_id'].nunique().to_dict()
    print(f"   {split_name}: {counts}  total={sum(counts.values())}")

# ---------- 4. Save (ALL subsets combined) ----------
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

full_train_df.to_csv(PROCESSED_DIR / 'train_FD001.csv', index=False)
full_test_df.to_csv(PROCESSED_DIR  / 'test_FD001.csv',  index=False)
full_train_df.to_csv(INTERIM_DATA_DIR / 'train_with_rul.csv', index=False)
full_test_df.to_csv(INTERIM_DATA_DIR  / 'test_with_rul.csv',  index=False)

print(f"\n✅ Saved to {PROCESSED_DIR}:")
print(f"   train_FD001.csv  ({len(full_train_df):,} rows — ALL subsets)")
print(f"   test_FD001.csv   ({len(full_test_df):,} rows  — ALL subsets)")
print("   ⚠️  Filename says 'FD001' for backward compat, but contains FD001–FD004 combined.")


Train RUL stats:
                 RUL    RUL_clipped
count  160359.000000  160359.000000
mean      122.331338      90.182029
std        83.538146      41.241036
min         0.000000       0.000000
25%        56.000000      56.000000
50%       113.000000     113.000000
75%       172.000000     125.000000
max       542.000000     125.000000

Test RUL stats:
                 RUL    RUL_clipped
count  104897.000000  104897.000000
mean      162.628264     110.529195
std        80.873737      26.863275
min         6.000000       6.000000
25%       107.000000     107.000000
50%       154.000000     125.000000
75%       206.000000     125.000000
max       553.000000     125.000000

Feature Engineering Step 1 (RUL Labels) Complete.


# Section 5: Scaling, Validation Split & Sequence Generation

In [ ]:

# ==================== SCALING & SEQUENCE GENERATION (ALL 4 SUBSETS) ====================
print("\n--- SCALING & SEQUENCING (FD001 – FD004 COMBINED) ---")
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit
import joblib

# ---------- Helpers ----------
def get_feature_cols(df):
    """
    Returns columns suitable for scaling/modeling:
    - Excludes metadata identifiers and target columns.
    - Excludes constant columns (std == 0) to avoid NaN from MinMaxScaler.
    """
    exclude = {
        'dataset_id', 'engine_id', 'cycle', 'sub_dataset',
        'subset', 'RUL', 'RUL_clipped', 'max_cycle', 'RUL_end'
    }
    return [c for c in df.columns if c not in exclude and df[c].std() > 0]

# ---------- Regime-based scaler for multi-condition subsets ----------
def fit_regime_scaler(train_df, sensor_cols, n_regimes=6):
    """
    Cluster operating points via KMeans on op_settings, then fit one
    MinMaxScaler per regime.  Returns (km_model, {regime_id: scaler}).
    """
    km = KMeans(n_clusters=n_regimes, random_state=42, n_init=10)
    km.fit(train_df[['op_setting_1', 'op_setting_2', 'op_setting_3']])
    scalers = {}
    for regime_id in range(n_regimes):
        mask = km.labels_ == regime_id
        sc = MinMaxScaler(feature_range=(-1, 1))
        sc.fit(train_df.loc[mask, sensor_cols])
        scalers[regime_id] = sc
    return km, scalers

def apply_regime_scaler(df, km, scalers, sensor_cols):
    """Predict regime for each row, then scale with the per-regime scaler."""
    df = df.copy()
    labels = km.predict(df[['op_setting_1', 'op_setting_2', 'op_setting_3']])
    for regime_id, sc in scalers.items():
        mask = labels == regime_id
        if mask.any():
            df.loc[mask, sensor_cols] = sc.transform(df.loc[mask, sensor_cols])
    return df

# ---------- Scale each subset with its appropriate strategy ----------
# FD001 (id=1) and FD003 (id=3): 1 operating condition → global MinMaxScaler
# FD002 (id=2) and FD004 (id=4): 6 operating conditions → KMeans regime scaler

MULTI_COND_SUBSETS = {2, 4}
N_REGIMES = 6

train_featured = full_train_df.copy()
test_featured  = full_test_df.copy()

scaler_store = {}          # persisted for inference
train_scaled_pieces = []
test_scaled_pieces  = []
subset_feature_cols = {}   # track per-subset columns

print(f"📊 Scaling {sorted(train_featured['dataset_id'].unique())} subsets:")

for did in sorted(train_featured['dataset_id'].unique()):
    tr = train_featured[train_featured['dataset_id'] == did].copy()
    te = test_featured[test_featured['dataset_id'] == did].copy()
    fcols = get_feature_cols(tr)
    subset_feature_cols[did] = fcols

    if did in MULTI_COND_SUBSETS:
        # KMeans regime-based scaling — preserves inter-regime sensor ranges
        km, scalers = fit_regime_scaler(tr, fcols, n_regimes=N_REGIMES)
        tr = apply_regime_scaler(tr, km, scalers, fcols)
        te = apply_regime_scaler(te, km, scalers, fcols)
        scaler_store[did] = {
            'type': 'regime', 'km': km, 'scalers': scalers, 'feature_cols': fcols
        }
        print(f"   dataset_id={did}: KMeans regime-based scaling "
              f"({N_REGIMES} regimes, {len(fcols)} features, "
              f"{len(tr)} train + {len(te)} test rows)")
    else:
        # Single global scaler for single-condition subsets
        sc = MinMaxScaler(feature_range=(-1, 1))
        tr[fcols] = sc.fit_transform(tr[fcols])
        te[fcols] = sc.transform(te[fcols])
        scaler_store[did] = {'type': 'global', 'scaler': sc, 'feature_cols': fcols}
        print(f"   dataset_id={did}: global MinMaxScaler "
              f"({len(fcols)} features, {len(tr)} train + {len(te)} test rows)")

    train_scaled_pieces.append(tr)
    test_scaled_pieces.append(te)

train_featured = pd.concat(train_scaled_pieces, ignore_index=True)
test_featured  = pd.concat(test_scaled_pieces,  ignore_index=True)

# feature_cols for sequence generation = intersection across all subsets
# (guarantees every subset has a value for every feature — no NaN-padded sequences)
all_fcols_sets = [set(v) for v in subset_feature_cols.values()]
feature_cols = sorted(set.intersection(*all_fcols_sets))
print(f"\n✅ Shared feature columns for sequence generation: {len(feature_cols)}")

# Save all scalers in one file
MODELS_DIR = PROJECT_ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
joblib.dump(scaler_store, MODELS_DIR / 'feature_scaler.joblib')
print(f"✅ Scaler store saved to {MODELS_DIR / 'feature_scaler.joblib'}")

# ---------- Validation Split — GroupShuffleSplit keeps ALL cycles of each engine together ----------
# Uses composite key (dataset_id + engine_id) so FD001 engine 1 ≠ FD002 engine 1
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
grp = train_featured[['dataset_id', 'engine_id']].apply(
    lambda r: f"{r.dataset_id}_{r.engine_id}", axis=1)
tr_idx, va_idx = next(gss.split(train_featured, groups=grp))
train_split = train_featured.iloc[tr_idx].copy()
val_split   = train_featured.iloc[va_idx].copy()

# Verify no engine leakage
tr_keys = set(train_split['dataset_id'].astype(str) + '_' + train_split['engine_id'].astype(str))
va_keys = set(val_split['dataset_id'].astype(str) + '_' + val_split['engine_id'].astype(str))
assert len(tr_keys & va_keys) == 0, "❌ ENGINE LEAKAGE between train/val!"
print(f"\n✅ Train/Val Split (no engine leakage):")
print(f"   Train: {train_split.shape}  engines={len(tr_keys)}")
print(f"   Val:   {val_split.shape}    engines={len(va_keys)}")
print(f"   Test:  {test_featured.shape}")

# Save processed CSVs — these contain ALL 4 subsets combined
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
train_split.to_csv(PROCESSED_DIR / 'train_FD001.csv',  index=False)
val_split.to_csv(  PROCESSED_DIR / 'val_FD001.csv',    index=False)
test_featured.to_csv(PROCESSED_DIR / 'test_FD001.csv', index=False)

# Verify all subsets present in output
for name, df_ in [('train_FD001.csv', train_split), ('val_FD001.csv', val_split), ('test_FD001.csv', test_featured)]:
    subs = sorted(df_['subset'].unique()) if 'subset' in df_.columns else ['unknown']
    print(f"   💾 {name}: {len(df_):,} rows — subsets: {subs}")
print("   ℹ️  Filenames say 'FD001' for backward compat, but contain ALL subsets (FD001–FD004).")

# Alias for sequence generation below
X_train_df = train_split
X_val_df   = val_split

# ---------- Sequence Generation Function (Sliding Window) ----------
def create_sequences(df, feature_cols, sequence_length=30, pad=True):
    """
    Creates (samples, seq_len, features) tensor from ALL subsets combined.
    Uses composite key (dataset_id + engine_id) so windows don't cross engines.
    If pad=True, engines shorter than seq_len are pre-padded with 0s.
    Target is the RUL of the LAST step in the window.
    """
    X_seq = []
    y_seq = []

    for _, group in df.groupby(['dataset_id', 'engine_id']):
        features = group[feature_cols].values
        target   = group['RUL_clipped'].values
        num_cycles = len(group)

        if num_cycles >= sequence_length:
            for i in range(num_cycles - sequence_length + 1):
                X_seq.append(features[i : i + sequence_length])
                y_seq.append(target[i + sequence_length - 1])
        elif pad:
            pad_len  = sequence_length - num_cycles
            padding  = np.zeros((pad_len, features.shape[1]))
            X_window = np.vstack([padding, features])
            X_seq.append(X_window)
            y_seq.append(target[-1])

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32)

print("\n⏳ Generating sequences (Window=30) for ALL subsets...")
SEQ_LEN = 30

X_train_seq, y_train_seq = create_sequences(X_train_df,   feature_cols, SEQ_LEN, pad=False)
X_val_seq,   y_val_seq   = create_sequences(X_val_df,     feature_cols, SEQ_LEN, pad=False)
X_test_seq,  y_test_seq  = create_sequences(test_featured, feature_cols, SEQ_LEN, pad=True)

print(f"✅ Train Sequences: {X_train_seq.shape} | Labels: {y_train_seq.shape}")
print(f"✅ Val Sequences:   {X_val_seq.shape}   | Labels: {y_val_seq.shape}")
print(f"✅ Test Sequences:  {X_test_seq.shape}   | Labels: {y_test_seq.shape}")

SEQ_DIR = PROCESSED_DIR / 'sequences'
SEQ_DIR.mkdir(exist_ok=True)
np.save(SEQ_DIR / 'X_train_seq.npy', X_train_seq)
np.save(SEQ_DIR / 'y_train_seq.npy', y_train_seq)
np.save(SEQ_DIR / 'X_val_seq.npy',   X_val_seq)
np.save(SEQ_DIR / 'y_val_seq.npy',   y_val_seq)
np.save(SEQ_DIR / 'X_test_seq.npy',  X_test_seq)
np.save(SEQ_DIR / 'y_test_seq.npy',  y_test_seq)
print(f"✅ Saved sequences to {SEQ_DIR} (ALL subsets combined)")

print(f"\n🎉 NB01 COMPLETE — processed ALL 4 C-MAPSS subsets (FD001–FD004).")
print(f"   Total engines: ~708 | Total rows: ~{len(full_train_df) + len(full_test_df):,}")



--- SCALING & SEQUENCING ---
   dataset_id=1: global MinMaxScaler (19 features)
   dataset_id=2: KMeans regime-based scaling (6 regimes, 24 features)
   dataset_id=3: global MinMaxScaler (20 features)
   dataset_id=4: KMeans regime-based scaling (6 regimes, 24 features)

✅ Shared feature columns for sequence generation: 19
✅ Scaler store saved to /Users/xe/models/feature_scaler.joblib

Split Engines: 567 Train, 142 Val
✅ Saved: train=(129029, 31), val=(31330, 31), test=(104897, 33)

Generating sequences (Window=30)...
✅ Train Sequences: (112586, 30, 19) | Labels: (112586,)
✅ Val Sequences:   (27212, 30, 19)   | Labels: (27212,)
✅ Test Sequences:  (84495, 30, 19)   | Labels: (84495,)
✅ Saved Numpy Sequences to /Users/xe/data/processed/sequences


# Section 6: Download and Select LogHub Datasets

## LogHub Datasets

LogHub is a collection of public log datasets from real systems:

| Dataset | System | Size | Entries | Characteristics |
|---------|--------|------|---------|----------|
| HDFS | Hadoop | ~55 MB | 11M | Distributed file system logs |
| BGL | Blue Gene/L | ~700 MB | 4.7M | Supercomputer logs |
| OpenStack | Cloud | ~300 MB | 207k | Cloud infrastructure logs |
| Android | Mobile | Large | 1.5M | Mobile OS logs |

We'll focus on **HDFS** and **BGL** for this project.

# Section 7: Normalize and Parse Log Data

## Example: Parsing synthetic log data

Since we don't have the actual LogHub data yet, we'll demonstrate the parsing pipeline with example logs.

# Section 8: Convert Logs to Incident Narratives

# Section 9: Store Clean Text Corpus


# Summary: PHASE 2 Complete ✅

## What We Accomplished:

### Time-Series Data (NASA C-MAPSS — ALL 4 Subsets)
- ✅ Loaded and parsed **ALL 4 C-MAPSS subsets** (FD001–FD004):
  - **FD001**: 100 engines, 1 op condition, 1 fault mode
  - **FD002**: 260 engines, 6 op conditions, 1 fault mode
  - **FD003**: 100 engines, 1 op condition, 2 fault modes
  - **FD004**: 248 engines, 6 op conditions, 2 fault modes
- ✅ Combined into single DataFrames with `subset` + `dataset_id` columns
- ✅ Applied scaling strategy per subset:
  - FD001/FD003: Global MinMaxScaler (single operating condition)
  - FD002/FD004: KMeans regime-based scaling (6 operating conditions)
- ✅ Created RUL labels (piecewise linear, clipped at 125)
- ✅ Split train/val by engine using GroupShuffleSplit (80/20, no leakage)
- ✅ Generated sliding-window sequences (length=30) for deep learning

### Text Data (LogHub)
- ✅ Demonstrated log parsing pipeline
- ✅ Normalized log messages, grouped into incident bursts
- ✅ Generated incident narratives & synthetic maintenance reports
- ✅ Stored clean text corpus

## Outputs:

**Time-Series Data (ALL subsets combined):**
- `data/processed/train_FD001.csv` — Training data (80% of engines from ALL 4 subsets)
- `data/processed/val_FD001.csv` — Validation data (20% of engines from ALL 4 subsets)
- `data/processed/test_FD001.csv` — Test data (ALL 4 subsets, separate engine set)
- `data/processed/sequences/` — Numpy sequences for LSTM/TCN (ALL subsets)
- `models/feature_scaler.joblib` — Per-subset scalers
- `reports/figures/eda/` — Sensor visualizations, correlation heatmaps

**Text Data:**
- `data/interim/combined_text_corpus.csv` — Parsed logs
- `data/processed/text_corpus/` — Cleaned logs, incidents, narratives

## Next Steps (PHASE 3+):
- Feature engineering pipeline (rolling aggregates, degradation signals)
- Baseline 1: XGBoost + Random Forest on ALL subsets
- Baseline 2: LSTM + TCN deep learning models
- Anomaly detection, RAG pipeline, Agentic AI orchestration
